In [14]:
#collect here all imports and installs
import cv2
import os
import subprocess
import numpy as np
import pandas as pd
import time



#see what we need from these 
import pandas as pd
import numpy as np
import re
import urllib.parse


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn import metrics
from sklearn.metrics import accuracy_score 



import matplotlib.pyplot as plt
from sklearn import tree
from six import StringIO  
from IPython.display import Image  
import pydotplus
from matplotlib.colors import LinearSegmentedColormap
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# assign directory
directory = 'videos'

# Start the timer
start_time = time.time()
df = CreateFeatureTable(directory)
print(df)
# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print(f"Table created in: {elapsed_time} seconds")

In [11]:
df

,Name,Bitrate,width,height,spatial_info,temporal_info,fps,entropy
0,lessmotion.MOV,8693.462433,1920,1080,88.410443,118.711078,29.995115,6.870041
1,motion.MOV,8711.101114,1920,1080,62.280166,125.082111,29.989868,6.912870
2,1.mp4,22108.293169,4096,2160,111.269295,111.165728,30.000000,7.346967
3,raw.y4m,36460.374825,352,288,76.864047,93.444637,29.970030,7.251727
4,testscenes.MOV,5156.035241,1920,1080,12.568396,126.699359,29.994950,5.334857
5,1c.mp4,12237.673033,4096,2160,104.461582,105.318450,30.000000,7.331122
6,raw_c.mp4,520.186994,352,288,76.631422,114.270600,29.970000,7.184745
7,ProResc.mp4,80927.635266,3840,2160,61.564258,121.886149,29.900000,6.974097


In [ ]:
#without motion score
#Table created in: 188.67073893547058 seconds
    
#with spatial and temporal info
#Table created in: 629.0842838287354 seconds

#with motion score
#Table created in: 3010.119035959244 seconds


In [17]:
#Run to initiate functions 


def CreateFeatureTable(directory):
    #Create Pandas table 
    
    # Start the timer
    start_time = time.time()
    # Create the pandas DataFrame with column name is provided explicitly
    df = pd.DataFrame()
    
    filenames = []
    bitrates = [] 
    width = []
    height = []
    fps = [] #frames Per Second, should we round?
    
    average_intensity = []

    average_brightness = []
    average_contrast = [] 
    average_sharpness = []
    motion_score = []
    spatial_info = []
    temporal_info = []
    entropy = []


    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            filenames.append(f.replace('videos/', ''))
            b = GetBitrate(f)
            bitrates.append(b)
            #average_intensity.append(AverageSpatialValue(f))
            #SP = AverageSpatialInformation(f)
            #average_brightness.append(SP[0])
            #average_contrast.append(SP[1])
            #average_sharpness.append(SP[2])
            #motion_score.append(calculateMotionScore(f))
            
            properties = get_video_properties(f)
            width.append(properties[0])
            height.append(properties[1])
            fps.append(properties[2])
            spatial_info.append(calculate_spatial_information(f))
            temporal_info.append(calculate_temporal_information(f))
            entropy.append(calculate_video_entropy(f))
            

    df = df.assign(Name=filenames,
                   Bitrate=bitrates,
                   width=width,
                   height=height,
                   spatial_info=spatial_info,
                   temporal_info=temporal_info,
                   #average_intensity=average_intensity,
                   #average_brightness = average_brightness,
                   #average_contrast = average_contrast,
                   #average_sharpness = average_sharpness,
                   #motion_score = motion_score,
                   
                   fps=fps, #why is not working??
                   entropy=entropy
                  )
    # End the timer
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    print(f"Table created in: {elapsed_time} seconds")
  
    return df
    
#Find video properties 

def get_video_properties(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the video width and height
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Get the frames per second (FPS)
    fps = video.get(cv2.CAP_PROP_FPS)

    # Release the video file
    video.release()

    return width, height, fps


#Calculate the bitrate 

def GetBitrate(video_path):
    
    # Get video file size in bytes
    file_size = os.path.getsize(video_path)

    # Get video duration in seconds using FFprobe
    ffprobe_cmd = f'ffprobe -i "{video_path}" -show_entries format=duration -v quiet -of csv="p=0"'
    result = subprocess.run(ffprobe_cmd, capture_output=True, text=True, shell=True)
    duration = float(result.stdout.strip())

    # Calculate the estimated bitrate in kilobits per second (kbps)
    bitrate = (file_size * 8) / (duration * 1000)
    # Convert bitrate to megabits per second (Mbps)
    bitrate_mbps = bitrate / 1000
    
    
    return bitrate_mbps

def calculate_spatial_information(video_path):
    video = cv2.VideoCapture(video_path)
    sobel_stds = []

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply the Sobel filter to the luminance component
        sobel_x = cv2.Sobel(gray_frame, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(gray_frame, cv2.CV_64F, 0, 1, ksize=3)
        sobel = np.sqrt(sobel_x**2 + sobel_y**2)

        # Calculate the standard deviation of the Sobel filter
        sobel_std = np.std(sobel)

        sobel_stds.append(sobel_std)

    video.release()

    # Calculate the maximum standard deviation
    max_std = max(sobel_stds)

    return max_std

def calculate_temporal_information(video_path):
    video = cv2.VideoCapture(video_path)
    motion_diffs = []

    ret, prev_frame = video.read()

    while True:
        ret, curr_frame = video.read()

        if not ret:
            break

        # Convert frames to grayscale
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        # Calculate the motion difference between frames
        motion_diff = curr_gray - prev_gray

        # Calculate the standard deviation of the motion difference
        motion_std = np.std(motion_diff)

        motion_diffs.append(motion_std)

        prev_frame = curr_frame

    video.release()
    # Calculate the maximum standard deviation
    max_std = max(motion_diffs)

    return max_std


#WHAT IT GIVES US?



def calculate_entropy(image):
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist /= hist.sum()  # Normalize histogram

    probabilities = hist.flatten()
    probabilities = probabilities[probabilities != 0]  # Remove zero probabilities

    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

def calculate_video_entropy(video_path):
    cap = cv2.VideoCapture(video_path)
    entropy_values = []

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        entropy = calculate_entropy(gray_frame)
        entropy_values.append(entropy)

    cap.release()

    average_entropy = np.mean(entropy_values)
    return average_entropy


#Average Spatial Value

def AverageSpatialValue(video_path):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is successfully opened
    if not cap.isOpened():
        print("Error opening video file")
        exit()

    # Initialize variables
    frame_count = 0
    total_intensity = 0

    # Read and process frames
    while True:
        # Read a frame from the video
        ret, frame = cap.read()

        # If the frame was not read successfully, then we have reached the end of the video
        if not ret:
            break

        # Calculate the average pixel intensity of the frame
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        intensity = np.mean(gray_frame)

        # Accumulate the total intensity
        total_intensity += intensity

        # Increment frame count
        frame_count += 1

    # Calculate the average spatial value
    average_intensity = total_intensity / frame_count

    # Release the video capture object and close any open windows
    cap.release()
    cv2.destroyAllWindows()
    
    return average_intensity

#Calculate average spatial information


import cv2
def AverageSpatialInformation(video_path):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Initialize variables to store spatial information
    frame_count = 0
    total_brightness = 0.0
    total_contrast = 0.0
    total_sharpness = 0.0

    # Iterate over the frames
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If no more frames, break the loop
        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate brightness (average pixel intensity)
        brightness = gray_frame.mean()
        total_brightness += brightness
        #print('calculate brightness')
        # Calculate contrast (standard deviation of pixel intensities)
        contrast = gray_frame.std()
        total_contrast += contrast
        #print('calculate contrast')
        # Calculate sharpness (Laplacian variance)
        laplacian_var = cv2.Laplacian(gray_frame, cv2.CV_64F).var()
        total_sharpness += laplacian_var
        #print('calculate sharpness')

        # Increment frame count
        frame_count += 1

    # Release the video capture
    cap.release()

    # Calculate average spatial information
    average_brightness = total_brightness / frame_count
    average_contrast = total_contrast / frame_count
    average_sharpness = total_sharpness / frame_count
    
    return average_brightness, average_contrast, average_sharpness

    #print(frame_count)
    # Print the spatial information measures
    print(f"Average Brightness: {average_brightness}")
    print(f"Average Contrast: {average_contrast}")
    print(f"Average Sharpness: {average_sharpness}")



In [ ]:
#TAKES TOO LONG

#Analyse motion
#Returns video Motion Score


def calculateMotionScore(video_path):

    # Start the timer
    #start_time = time.time()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, prev_frame = cap.read()

    # Convert the frame to grayscale
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Initialize motion vectors
    motion_vectors = []

    # Set parameters for Farneback optical flow
    params = dict(pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)

    # Iterate over the frames
    while True:
        # Read the next frame
        ret, frame = cap.read()

        # If no more frames, break the loop
        if not ret:
            break

        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow using Farneback algorithm
        flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, frame_gray, None, **params)

        # Calculate the magnitude of motion vectors
        magnitude = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)

        # Calculate the average magnitude
        average_magnitude = np.mean(magnitude)

        # Store the average magnitude as a motion value
        motion_vectors.append(average_magnitude)

        # Set the current frame as the previous frame for the next iteration
        prev_frame_gray = frame_gray.copy()

    # Release the video capture
    cap.release()

    # Calculate the overall motion score
    motion_score = np.mean(motion_vectors)



    # End the timer
    #end_time = time.time()

    # Calculate the elapsed time
    #elapsed_time = end_time - start_time

    #print(f"Elapsed Time too find motion score: {elapsed_time} seconds")

    #print(f"Motion Score: {motion_score}")
    return motion_score


In [ ]:

#for working with csv from Leon 

path_to_dataset = 'features.csv'
path_to_labels = 'bitrates.csv'

#upload 
#upload data
df_features = pd.read_csv(path_to_dataset)
df_labels = pd.read_csv(path_to_labels)
#cut names of videos
#this works only for current table, have to think how we work with names later

#df
#df_labels big buck bunny 720p24 Scene 049.mp4
df_features['Name'] = df_features['Name'].str[-7:]
df_labels['Name'] = df_labels['Name'].str[-7:]

#drop first column df_labels
df_labels = df_labels.drop(columns=['Unnamed: 0'])


#merge tables 
df = pd.merge(df_features, df_labels, on=['Name'])
df = df.rename(columns={'Bitrate_y': 'Bitrate_Label'}) 
df.head()


In [ ]:
#DECISION TREE

# import the regressor
from sklearn.tree import DecisionTreeRegressor 


#assume df is a table with features and labels 
#where label is in the last column called Bitrate_Label

#names of the columns with features, 
#can also just take all columns apart from first with name and last with label

#we can leave some out of course
#list_of_features = ['Bitrate_x', 'spatial_info','temporal_info', 'average_intensity',
#                   'average_brightness', 'average_contrast', 'average_sharpness']

list_of_features = ['Bitrate_x', 'spatial_info','temporal_info']

X = df[list_of_features] #Features
y = df.Bitrate_Label #Labels



#split the data into train and test 

#Split the data into training set and test set
# 67% training and 33% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 


# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X_train, y_train)

# Predict

y_predicted = regressor.predict(X_test)


#to compare the result we have to set some treshhold 
#like +-5% of abweichung in bitrate for prediction 
compare = pd.DataFrame()
compare= compare.assign(test =y_test, predicted = y_predicted)


In [ ]:
compare

In [ ]:
#try changing to mp4 with Leon#s code
import os
import subprocess
import pandas as pd
import numpy as np
import re
import math
import time
#import MediaInfo
import shutil


#do qp0 encode

#folder
orig_scenes_loc = 'videos'

#name of thefile we want to turn intomp4
filename = 'blue_sky.y4m'
input_vid_res = '1920x1080'
input_vid_fps = '25'

input_vid_pix_fmt = 'yuv420p'


#input_vid_name  =
#scene_encode_loc = 

#name of the compresed file
qp0_filename = f'videos/qp0_blue_sky.mp4'

qp0_command = f'ffmpeg -v error -f rawvideo -vcodec rawvideo -s {input_vid_res} -r {input_vid_fps} -pix_fmt {input_vid_pix_fmt} -i {orig_scenes_loc}/{filename} -c:v libx264 -preset ultrafast -qp 0 {qp0_filename}'
qp0_result = subprocess.run(qp0_command, capture_output=True, text=True, shell=True)
print('QP0 RESULT: ', qp0_result)
